In [1]:
import os

In [2]:
os.chdir("E:\\datascienceproject")
%pwd

'E:\\datascienceproject'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str


In [5]:
from src.datascienece.constants import *
from src.datascienece.utils.common import read_yaml, create_directories


In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH,
                 schema_file_path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
           root_dir=config.root_dir,
           train_data_path=config.train_data_path,
           test_data_path=config.test_data_path,
           model_name=config.model_name,
           alpha=params.alpha,
           l1_ratio=params.l1_ratio,
          target_column=self.schema.TARGET_COLUMN.name)
           
        return model_trainer_config
  

In [8]:
import pandas as pd
import os
from src.datascienece import logger
from sklearn.linear_model import ElasticNet
import joblib

In [12]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    def train_model(self):
        try:
            logger.info("Reading training and testing data")
            train_data = pd.read_csv(self.config.train_data_path)
            test_data = pd.read_csv(self.config.test_data_path)
            
            logger.info("Splitting data into features and target")
            X_train = train_data.drop(columns=[self.config.target_column], axis=1)
            y_train = train_data[self.config.target_column]
            
            X_test = test_data.drop(columns=[self.config.target_column])
            y_test = test_data[self.config.target_column]
            
            logger.info("Training the model")
            model = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio,random_state=42)
            model.fit(X_train, y_train)
            
            logger.info("Saving the model")
            model_path = os.path.join(self.config.root_dir, self.config.model_name)
            joblib.dump(model, model_path)
            
        except Exception as e:
            logger.exception(f"Error in training the model: {e}")

In [13]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train_model()
except Exception as e:
    raise e

[2026-02-24 16:27:05,944: INFO: common]: yaml file: config\config.yaml loaded successfully
[2026-02-24 16:27:05,946: INFO: common]: yaml file: parameters.yaml loaded successfully
[2026-02-24 16:27:05,948: INFO: common]: yaml file: schema.yaml loaded successfully
[2026-02-24 16:27:05,949: INFO: common]: Directory created at: artifacts
[2026-02-24 16:27:05,950: INFO: common]: Directory created at: artifacts/model_trainer
[2026-02-24 16:27:05,951: INFO: 706032753]: Reading training and testing data
[2026-02-24 16:27:05,956: INFO: 706032753]: Splitting data into features and target
[2026-02-24 16:27:05,959: INFO: 706032753]: Training the model
[2026-02-24 16:27:05,971: INFO: 706032753]: Saving the model
